# Zadání:
## Analýza plánovaných hodin na učitele (ve skupině – např. katedra/fakulta) v daném časovém období (gql_ug + gql_events)

- Vytvořit GQL dotaz na základě existující federace,
- Definovat transformaci GQL response -> table rows (vstup pro kontingenční tabulku)
- Vytvořit kontingenční tabulku
- Vytvořit koláčový / sloupcový graf
- Vytvořit Sunburst / Chord graf

Výsledek realizujte jako ipynb notebook (autentizace jménem a heslem, realizace aiohttp, transformace response, vytvoření tabulky, vytvoření grafu).

### Instalace potřebných knihoven
- **pandas** - Slouží k vytváření kontingenční tabulky
- **aiohttp a asyncio** - Umožňuje asynchronní funkce 
- **plotly** - Knihovna sloužící k vytváření koláčového a Sunburst grafu
- **aiofiles** - Umožňuje asynchronní manipulaci se soubory (Využívá se z důvodu práce s "vlastními" daty)

In [59]:
%pip install pandas aiohttp plotly.express asyncio nbformat aiofiles datetime

Note: you may need to restart the kernel to use updated packages.


### Import pořebných knihoven

In [74]:
import aiohttp
import asyncio
import json
import plotly.express as px
import pandas as pd
from itertools import product
from functools import reduce
import aiofiles
from typing import List, Union, Dict
import datetime
from collections import defaultdict

### Funkce pro získání tokenu


In [61]:
async def getToken(username, password):
    keyurl = "http://localhost:33001/oauth/login3"
    async with aiohttp.ClientSession() as session:
        async with session.get(keyurl) as resp:
            keyJson = await resp.json()

        payload = {"key": keyJson["key"], "username": username, "password": password}
        async with session.post(keyurl, json=payload) as resp:
            tokenJson = await resp.json()
    return tokenJson.get("token", None)

### Funkce pro definici GraphQL dotazu

In [62]:
def query(q, token):
    async def post(variables):
        gqlurl = "http://localhost:33001/api/gql"
        payload = {"query": q, "variables": variables}
        cookies = {'authorization': token}
        async with aiohttp.ClientSession() as session:
            async with session.post(gqlurl, json=payload, cookies=cookies) as resp:
                if resp.status != 200:
                    text = await resp.text()
                    print(text)
                    return text
                else:
                    response = await resp.json()
                    return response
    return post

### Funkce pro zpracování dat

In [63]:
def enumerateAttrs(attrs):
    for key, value in attrs.items():
        names = value.split(".")
        name = names[0]
        yield key, name

def flattenList(inList, outItem, attrs):
    for item in inList:
        assert isinstance(item, dict), f"in list only dicts are expected"
        for row in flatten(item, outItem, attrs):
            yield row

def flattenDict(inDict, outItem, attrs):
    result = {**outItem}
    complexAttrs = []
    for key, value in enumerateAttrs(attrs):
        attributeValue = inDict.get(value, None)
        if isinstance(attributeValue, list):
            complexAttrs.append((key, value))
        elif isinstance(attributeValue, dict):
            complexAttrs.append((key, value))
        else:
            result[key] = attributeValue
    lists = []
    for key, value in complexAttrs:
        attributeValue = inDict.get(value, None)
        prefix = f"{value}."
        prefixlen = len(prefix)
        subAttrs = {key: value[prefixlen:] for key, value in attrs.items() if value.startswith(prefix)}
        items = list(flatten(attributeValue, result, subAttrs))
        lists.append(items)
                     
    if len(lists) == 0:
        yield result
    else:
        for element in product(*lists):
            reduced = reduce(lambda a, b: {**a, **b}, element, {})
            yield reduced

def flatten(inData, outItem, attrs):
    if isinstance(inData, dict):
        for item in flattenDict(inData, outItem, attrs):
            yield item
    elif isinstance(inData, list):
        for item in flattenList(inData, outItem, attrs):
            yield item
    else:
        assert False, f"Unexpected type on inData {inData}"

### Přihlašovací údaje, GQL dotaz, Mappers

**Přihlašovací údaje**
Přihlašovací údaje jsou potřebné k získání tokenu, dostupná na frontend/graphiql
**Dotaz**
V dotazu využíváme groupPage, ze kterého extrahujeme membership a jeho group, user a jeho events

1. group
    - id - slouží k identifikaci do jaké skupiny uživatel patří (K-209, K-210, 23-5KB, ...)
    - name - slouží k následnému zobrazení skupiny v Sunburst diagramu
2. user
    - id - slouží k identifikaci jednotlivých uživatelů
    - fullname - slouží k zobrazení dat v grafech
3. events *využit fiter **where** pro získání dat v učitém datovém úseku*
    - id - pro identifikaci jednotlivých předmětů
    - name - Využívá se při zobraování dat v grafech
    - startdate - Nevyužito (Vloženo pro případ potřeby práce s časem)
    - enddate - Nevyužito (Vloženo pro případ potřeby práce s časem)
4. presences
    - id - součtem získáme počet hodin, na kterých byl přítomen (následně vynásobíme 90 - vyučovací blok)
5. InvitationType
    - id - Určujeme, zda se jedná o organizátora (Vyučujícícho)

In [70]:
username = "john.newbie@world.com"
password = "john.newbie@world.com"

queryStr = """
{
  groupPage {
    memberships {
      group {
        id
        name
      }
      user {
        id
        fullname
        events(
          where: {_and: [{startdate: {_ge: "2023-03-19T08:00:00"}}, {enddate: {_le: "2023-05-19T08:00:00"}}]}
        ) {
          id
          name
          startdate
          enddate
          presences {
            id
            invitationType {
              id
            }
          }
        }
      }
    }
  }
}
"""

mappers = {
    "groupID": "memberships.group.id",
    "groupName": "memberships.group.name",
    "userID": "memberships.user.id",
    "userFullname": "memberships.user.fullname",
    "eventID": "memberships.user.events.id",
    "eventName": "memberships.user.events.name",
    "startdate": "memberships.user.events.startdate",
    "eventEndDate": "memberships.user.events.enddate",
    "presenceID": "memberships.user.events.presences.id",
    "invitationTypeID": "memberships.user.events.presences.invitationType.id",
}

In [71]:
async def fullPipe():
    token = await getToken(username, password)
    qfunc = query(queryStr, token)
    response = await qfunc({})
    print(response)
    data = response.get("data", None)
    
    # Debugging statement to inspect the structure
    print("Data:", data)
    
    # Check if data is not None and is a dictionary
    if not data or not isinstance(data, dict):
        raise ValueError("Invalid response data structure")
    
    groupPage = data.get("groupPage")
    
    # Ensure groupPage is a dictionary and extract memberships
    memberships = []
    if isinstance(groupPage, list) and len(groupPage) > 0:
        memberships = groupPage[0].get("memberships", [])
    elif isinstance(groupPage, dict):
        memberships = groupPage.get("memberships", [])

    # Flatten and map the data using list comprehensions
    resultMapped = []
    for membership in memberships:
        group = membership.get("group", {})
        user = membership.get("user", {})
        events = user.get("events", [])
        for event in events:
            presences = event.get("presences", [])
            for presence in presences:
                resultMapped.append({
                    "groupID": group.get("id"),
                    "groupName": group.get("name"),
                    "userID": user.get("id"),
                    "userFullname": user.get("fullname"),
                    "eventID": event.get("id"),
                    "eventName": event.get("name"),
                    "startdate": event.get("startdate"),
                    "eventEndDate": event.get("enddate"),
                    "invitationTypeID": presence.get("invitationType", {}).get("id")
                })

    return resultMapped

async def main():
    flatData = await fullPipe()
    print(flatData)
    with open('resultQuery.json', "w", encoding='utf-8') as outputFile:
        json.dump(flatData, outputFile)
        print("json Dumped")

await main()


{'data': {'groupPage': [{'memberships': [{'group': {'id': '2d9dcd22-a4a2-11ed-b9df-0242ac120003', 'name': 'Uni'}, 'user': {'id': '2d9dc5ca-a4a2-11ed-b9df-0242ac120003', 'fullname': 'John Newbie', 'events': []}}, {'group': {'id': '2d9dcd22-a4a2-11ed-b9df-0242ac120003', 'name': 'Uni'}, 'user': {'id': '2d9dc868-a4a2-11ed-b9df-0242ac120003', 'fullname': 'Julia Newbie', 'events': []}}, {'group': {'id': '2d9dcd22-a4a2-11ed-b9df-0242ac120003', 'name': 'Uni'}, 'user': {'id': '2d9dc9a8-a4a2-11ed-b9df-0242ac120003', 'fullname': 'Johnson Newbie', 'events': []}}, {'group': {'id': '2d9dcd22-a4a2-11ed-b9df-0242ac120003', 'name': 'Uni'}, 'user': {'id': '2d9dcbec-a4a2-11ed-b9df-0242ac120003', 'fullname': 'Jepeto Newbie', 'events': []}}, {'group': {'id': '2d9dcd22-a4a2-11ed-b9df-0242ac120003', 'name': 'Uni'}, 'user': {'id': '89d1e724-ae0f-11ed-9bd8-0242ac110002', 'fullname': 'Jana Newbie', 'events': [{'id': '45b2df80-ae0f-11ed-9bd8-0242ac110002', 'name': 'Zkouška', 'startdate': '2023-04-19T08:00:00', '

<h2>Tvorba tabulky</h2>

In [72]:
with open("resultQuery.json", "r") as file: 
    data = json.load(file)

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
df = pd.DataFrame(data)

print(df)

                                 groupID groupName                                userID userFullname                               eventID eventName            startdate         eventEndDate                      invitationTypeID
0   2d9dcd22-a4a2-11ed-b9df-0242ac120003       Uni  89d1e724-ae0f-11ed-9bd8-0242ac110002  Jana Newbie  45b2df80-ae0f-11ed-9bd8-0242ac110002   Zkouška  2023-04-19T08:00:00  2023-04-19T09:00:00  e871403c-a79c-11ed-b76e-0242ac110002
1   2d9dcd22-a4a2-11ed-b9df-0242ac120003       Uni  89d1e724-ae0f-11ed-9bd8-0242ac110002  Jana Newbie  45b2df80-ae0f-11ed-9bd8-0242ac110002   Zkouška  2023-04-19T08:00:00  2023-04-19T09:00:00  e871403c-a79c-11ed-b76e-0242ac110002
2   2d9dcd22-a4a2-11ed-b9df-0242ac120003       Uni  89d1e724-ae0f-11ed-9bd8-0242ac110002  Jana Newbie  45b2df80-ae0f-11ed-9bd8-0242ac110002   Zkouška  2023-04-19T08:00:00  2023-04-19T09:00:00  e871403c-a79c-11ed-b76e-0242ac110002
3   2d9dcd22-a4a2-11ed-b9df-0242ac120003       Uni  89d1e724-ae0f-11ed-9bd8-0242

<h2>Tvorba grafů</h2>

### Funkce pro filtrování dat pro speciální skupinu

- Tuto funkci lze implementovat do kódu, který vykresluje koláčový graf (uvedena zvlášť pro větší přehlednost)
- Dochází k nahrání souboru fakeData.json (slouží pro lepší visualizaci daného problému v grafu)
- Zpracování json a filtrace na zakladě skupiny a organizatora


In [86]:
import json
import datetime

def load_json_data(path: str):
    with open(path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        return data

def user_attendance(data):
    source_table = []
    is_teacher = "e8713b6e-a79c-11ed-b76e-0242ac110002"  # Specifické ID po organizátora
    is_specific_group = "groupIdOfK209" # Specifické ID pro určitou skupinu
    
    for item in data:
        if item['invitationTypeID'] == is_teacher and item['groupID'] == is_specific_group:
            event_start = datetime.datetime.fromisoformat(item['startdate'])
            event_end = datetime.datetime.fromisoformat(item['eventEndDate'])

            row = {
                "userID": item['userID'],
                "userFullname": item['userFullname'],
                "eventID": item['eventID'],
                "eventName": item['eventName'],
                "invitationTypeID": item['invitationTypeID'],
                "lessonLength": (event_end - event_start).total_seconds() / 60,  # in minutes
            }
            source_table.append(row)

    print(source_table)

    
    with open('piechartData.json', "w", encoding='utf-8') as output_file:
        json.dump(source_table, output_file, ensure_ascii=False, indent=4)

def main():
    data = load_json_data('fakeData.json') # Pro implementaci je potřeba vyměnit resultFake za opravdová data z query
    if data:
        user_attendance(data)

if __name__ == "__main__":
    main()


[{'userID': '89d1e724-ae0f-11ed-9bd8-0242ac110002', 'userFullname': 'Jana Newbie', 'eventID': '45b2df80-ae0f-11ed-9bd8-0242ac110002', 'eventName': 'ABT', 'invitationTypeID': 'e8713b6e-a79c-11ed-b76e-0242ac110002', 'lessonLength': 60.0}, {'userID': '89d1e724-ae0f-11ed-9bd8-0242ac110002', 'userFullname': 'Jana Newbie', 'eventID': '45b2df80-ae0f-11ed-9bd8-0242ac110002', 'eventName': 'AIZ', 'invitationTypeID': 'e8713b6e-a79c-11ed-b76e-0242ac110002', 'lessonLength': 60.0}, {'userID': '89d1e724-ae0f-11ed-9bd8-0242ac110002', 'userFullname': 'Jana Newbie', 'eventID': '45b2df80-ae0f-11ed-9bd8-0242ac110002', 'eventName': 'AIZ', 'invitationTypeID': 'e8713b6e-a79c-11ed-b76e-0242ac110002', 'lessonLength': 90.0}, {'userID': '89d1e724-ae0f-11ed-9bd8-0242ac110002', 'userFullname': 'Jana Newbie', 'eventID': '45b2df80-ae0f-11ed-9bd8-0242ac110002', 'eventName': 'AIZ', 'invitationTypeID': 'e8713b6e-a79c-11ed-b76e-0242ac110002', 'lessonLength': 90.0}, {'userID': '29d1e724-ae0f-11ed-9bd8-0242ac110002', 'use

### Koláčový graf

- Koláčový graf využívá upravených dat pro lepší vizualizaci (resultFake)
- Graf zobrazuje počet odučených hodin na jednotlivé učitele v určité skupině za stanovené období.

In [87]:
with open('piechartData.json', 'r') as file:
    data = json.load(file)

def plot_lesson_lengths(data):
    aggregation = defaultdict(float)

    for entry in data:
        userFullname = entry['userFullname']
        lessonLength = entry['lessonLength']
        aggregation[userFullname] += lessonLength

    labels = list(aggregation.keys())
    values = list(aggregation.values())
    fig = px.pie(values=values, names=labels, title='Odučený počet hodin vyučujících na danné katedře')
    fig.show()

plot_lesson_lengths(data)

In [84]:
import json
import datetime

def load_json_data(path: str):
    with open(path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        return data

def user_attendance(data):
    source_table = []
    
    for item in data:
        event_start = datetime.datetime.fromisoformat(item['startdate'])
        event_end = datetime.datetime.fromisoformat(item['eventEndDate'])

        row = {
            "groupID": item['groupID'],
            "groupName": item['groupName'],
            "userID": item['userID'],
            "userFullname": item['userFullname'],
            "eventID": item['eventID'],
            "eventName": item['eventName'],
            "invitationTypeID": item['invitationTypeID'],
            "lessonLength": (event_end - event_start).total_seconds() / 60,  # in minutes
        }
        source_table.append(row)

    print(source_table)

    
    with open('sunburstData.json', "w", encoding='utf-8') as output_file:
        json.dump(source_table, output_file, ensure_ascii=False, indent=4)

def main():
    data = load_json_data('fakeData.json') # Pro implementaci je potřeba vyměnit resultFake za opravdová data z query
    if data:
        user_attendance(data)

if __name__ == "__main__":
    main()


[{'groupID': 'groupIdOfK209', 'groupName': 'K209', 'userID': '89d1e724-ae0f-11ed-9bd8-0242ac110002', 'userFullname': 'Jana Newbie', 'eventID': '45b2df80-ae0f-11ed-9bd8-0242ac110002', 'eventName': 'ABT', 'invitationTypeID': 'e8713b6e-a79c-11ed-b76e-0242ac110002', 'lessonLength': 60.0}, {'groupID': 'groupIdOfK209', 'groupName': 'K209', 'userID': '19d1e724-ae0f-11ed-9bd8-0242ac110002', 'userFullname': 'Jana BUBU', 'eventID': '45b2df80-ae0f-11ed-9bd8-0242ac110002', 'eventName': 'KYB', 'invitationTypeID': 'e444444c-a79c-11ed-b76e-0242ac110002', 'lessonLength': 60.0}, {'groupID': 'groupIdOfK209', 'groupName': 'K209', 'userID': '89d1e724-ae0f-11ed-9bd8-0242ac110002', 'userFullname': 'Jana Newbie', 'eventID': '45b2df80-ae0f-11ed-9bd8-0242ac110002', 'eventName': 'AIZ', 'invitationTypeID': 'e8713b6e-a79c-11ed-b76e-0242ac110002', 'lessonLength': 60.0}, {'groupID': 'groupIdOfK209', 'groupName': 'K209', 'userID': '89d1e724-ae0f-11ed-9bd8-0242ac110002', 'userFullname': 'Jana Newbie', 'eventID': '45b

In [85]:
# JSON data
with open('sunburstData.json', 'r') as file:
    data = json.load(file)

# Převod dat do pandas DataFrame
df = pd.DataFrame(data)

# Agregace dat pro získání celkového počtu 'presencesCount' podle 'groupID' a 'userFullname'
df_aggregated = df.groupby(['groupName', 'userFullname']).agg({'lessonLength': 'sum'}).reset_index()

# Vytvoření sunburst grafu
fig_sunburst = px.sunburst(
    df_aggregated,
    path=['groupName', 'userFullname', 'lessonLength'],
    values='lessonLength', 
    title='Přehled přítomnosti podle skupin a uživatelů'
)

# Aktualizace layoutu grafu
fig_sunburst.update_layout(
    width=800,  
    height=800 
)

fig_sunburst.show()